# Stroke pathway simulation - generation of results from alternative scenarios

This notebook runs alternative pathway simulations, with adjusted pathway parameters. The scenarios are:

1) Base: Uses the hospitals' recorded pathway statistics in SSNAP (same as validation notebook)

2) Speed: Sets 95% of patients having a scan within 4 hours of arrival, and all patients have 15 minutes arrival to scan and 15 minutes scan to needle.

3) Onset-known: Sets the proportion of patients with a known onset time of stroke to the national upper quartile if currently less than the national upper quartile (leave any greater than the upper national quartile at their current level).

4) Benchmark: The benchmark thrombolysis rate takes the likelihood to give thrombolysis for patients scanned within 4 hours of onset from the majority vote of the 30 hospitals with the highest predicted thrombolysis use in a standard 10k cohort set of patients. These are from Random Forests models.

5) Combine *Speed* and *Onset-known*

6) Combine *Speed* and *Benchmark*

7) Combine *Onset-known* and *Benchmark*

8) Combine *Speed* and *Onset-known* and *Benchmark*

Results are saved for each hospital and scenario. Detailed analysis will be performed in subsequent notebooks

## Import libraries and data

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathway import model_ssnap_pathway_scenarios

# load data from csv and store in pandas dataframe
filename = './hosp_performance_output/hospital_performance.csv'
hospital_performance_original = pd.read_csv(filename, index_col=0)

## Get results of alternative scenarios

### Base scenario

The base scenario uses the hospitals' recorded pathway statistics in SSNAP (same as validation notebook)

In [2]:
results_all = model_ssnap_pathway_scenarios(hospital_performance_original)
results_all['scenario'] = 'base'

# Save pathway stats used
hospital_performance_original.to_csv('output/performance_base.csv')

### Speed (30 minute arrival to needle)

The adjusted speed scenario sets 95% of patients having a scan within 4 hours of arrival, and all patients have 15 minutes arrival to scan and 15 minutes scan to needle.

In [3]:
# Create scenarios
hospital_performance = hospital_performance_original.copy()
hospital_performance['scan_within_4_hrs'] = 0.95
hospital_performance['arrival_scan_arrival_mins_mu'] = np.log(15)
hospital_performance['arrival_scan_arrival_mins_sigma'] = 0
hospital_performance['scan_needle_mins_mu'] = np.log(15)
hospital_performance['scan_needle_mins_sigma'] = 0

In [4]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'speed'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

# Save pathway stats used
hospital_performance.to_csv('output/performance_speed.csv')

### Known onset

Set the proportion of patients with a known onset time of stroke to the national upper quartile if currently less than the national upper quartile (leave any greater than the upper national quarile at their current level).

In [5]:
# Create scenarios
hospital_performance = hospital_performance_original.copy()

onset_known = hospital_performance_original['onset_known']
onset_known_upper_q = np.percentile(onset_known, 75)
adjusted_onset_known = []
for val in onset_known:
    if val > onset_known_upper_q:
        adjusted_onset_known.append(val)
    else:
        adjusted_onset_known.append(onset_known_upper_q)
hospital_performance['onset_known'] = adjusted_onset_known

In [6]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'onset'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

# Save pathway stats used
hospital_performance.to_csv('output/performance_onset.csv')

### Use benchmark thrombolysis

The benchmark thrombolysis rate takes the likelihood to give thrombolysis for patients scanned within 4 hours of onset from the majority vote of the 30 hospitals with the highest predicted thrombolysis use in a standard 10k cohort set of patients. These are from Random Forests models.

See Random Forests notebooks: *Benchmark hospitals* and *How would thrombolysis use change if clinical decisions were made by hospitals with the highest current thrombolysis rate?* 

In [7]:
# Load benchmark rates
filename = './hosp_performance_output/benchmark_4hr_scan.csv'
benchmark = pd.read_csv(filename, index_col=0)
# Convert from percentage to fraction
benchmark *= 0.01

In [8]:
# Merge in benchmark rates (to ensure order is correct)
hospital_performance = hospital_performance_original.copy()
hospital_performance = hospital_performance.merge(
    benchmark, left_index=True, right_index=True, how='left')
hospital_performance['eligable'] = hospital_performance['benchmark']

In [9]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'benchmark'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

# Save pathway stats used
hospital_performance.to_csv('output/performance_benchmark.csv')

### Combine speed and onset known

* 95% patients have scan within 4 hours of arrival

* 30 min arrival to needle (15 minute arrival-to-scan, 15 minute scan-to-needle)

* Proportion of patients with known stroke onset time set to upper quartile if currently lower

In [10]:
# Create scenarios
hospital_performance = hospital_performance_original.copy()

# Speed
hospital_performance['scan_within_4_hrs'] = 0.95
hospital_performance['arrival_scan_arrival_mins_mu'] = np.log(15)
hospital_performance['arrival_scan_arrival_mins_sigma'] = 0
hospital_performance['scan_needle_mins_mu'] = np.log(15)
hospital_performance['scan_needle_mins_sigma'] = 0

# Onset known
onset_known = hospital_performance_original['onset_known']
onset_known_upper_q = np.percentile(onset_known, 75)
adjusted_onset_known = []
for val in onset_known:
    if val > onset_known_upper_q:
        adjusted_onset_known.append(val)
    else:
        adjusted_onset_known.append(onset_known_upper_q)
hospital_performance['onset_known'] = adjusted_onset_known

# Save pathway stats used
hospital_performance.to_csv('output/performance_speed_onset.csv')

In [11]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'speed_onset'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

## Combine speed and benchmark

* 95% patients have scan within 4 hours of arrival

* 30 min arrival to needle (15 minute arrival-to-scan, 15 minute scan-to-needle)

* Decison to thrombolyse patients if scanned within 4 hours of known onset as predicted from majority vote of 30 benchmarl hospitals

In [12]:
# Create scenarios
hospital_performance = hospital_performance_original.copy()

# Speed
hospital_performance['scan_within_4_hrs'] = 0.95
hospital_performance['arrival_scan_arrival_mins_mu'] = np.log(15)
hospital_performance['arrival_scan_arrival_mins_sigma'] = 0
hospital_performance['scan_needle_mins_mu'] = np.log(15)
hospital_performance['scan_needle_mins_sigma'] = 0

# Benchmark
hospital_performance = hospital_performance.merge(
    benchmark, left_index=True, right_index=True, how='left')
hospital_performance['eligable'] = hospital_performance['benchmark']

In [13]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'speed_benchmark'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

# Save pathway stats used
hospital_performance.to_csv('output/performance_speed_benchmark.csv')

## Combine onset-known and benchmark

* Proportion of patients with known stroke onset time set to upper quartile if currently lower

* Decison to thrombolyse patients if scanned within 4 hours of known onset as predicted from majority vote of 30 benchmarl hospitals

In [14]:
# Create scenarios
hospital_performance = hospital_performance_original.copy()

# Onset known
onset_known = hospital_performance_original['onset_known']
onset_known_upper_q = np.percentile(onset_known, 75)
adjusted_onset_known = []
for val in onset_known:
    if val > onset_known_upper_q:
        adjusted_onset_known.append(val)
    else:
        adjusted_onset_known.append(onset_known_upper_q)
hospital_performance['onset_known'] = adjusted_onset_known

# Benchmark
hospital_performance = hospital_performance.merge(
    benchmark, left_index=True, right_index=True, how='left')
hospital_performance['eligable'] = hospital_performance['benchmark']

In [15]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'onset_benchmark'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

# Save pathway stats used
hospital_performance.to_csv('output/performance_onset_benchmark.csv')

## Combine speed, onset-known, and benchmark

* 95% patients have scan within 4 hours of arrival

* 30 min arrival to needle (15 minute arrival-to-scan, 15 minute scan-to-needle)

* Proportion of patients with known stroke onset time set to upper quartile if currently lower

* Decison to thrombolyse patients if scanned within 4 hours of known onset as predicted from majority vote of 30 benchmarl hospitals

In [16]:
# Create scenarios
hospital_performance = hospital_performance_original.copy()

# Speed
hospital_performance['scan_within_4_hrs'] = 0.95
hospital_performance['arrival_scan_arrival_mins_mu'] = np.log(15)
hospital_performance['arrival_scan_arrival_mins_sigma'] = 0
hospital_performance['scan_needle_mins_mu'] = np.log(15)
hospital_performance['scan_needle_mins_sigma'] = 0

# Onset known
onset_known = hospital_performance_original['onset_known']
onset_known_upper_q = np.percentile(onset_known, 75)
adjusted_onset_known = []
for val in onset_known:
    if val > onset_known_upper_q:
        adjusted_onset_known.append(val)
    else:
        adjusted_onset_known.append(onset_known_upper_q)
hospital_performance['onset_known'] = adjusted_onset_known

# Benchmark
hospital_performance = hospital_performance.merge(
    benchmark, left_index=True, right_index=True, how='left')
hospital_performance['eligable'] = hospital_performance['benchmark']

In [17]:
# Get results
results = model_ssnap_pathway_scenarios(hospital_performance)
results['scenario'] = 'speed_onset_benchmark'

# Add to results_all
results_all = pd.concat([results_all, results], axis=0)

# Save pathway stats used
hospital_performance.to_csv('output/performance_speed_onset_benchmark.csv')

In [18]:
results_all

,Baseline_good_outcomes_(median),Baseline_good_outcomes_per_1000_patients_(low_5%),Baseline_good_outcomes_per_1000_patients_(high_95%),Baseline_good_outcomes_per_1000_patients_(mean),Baseline_good_outcomes_per_1000_patients_(stdev),Baseline_good_outcomes_per_1000_patients_(95ci),Percent_Thrombolysis_(median%),Percent_Thrombolysis_(low_5%),Percent_Thrombolysis_(high_95%),Percent_Thrombolysis_(mean),...,Percent_Thrombolysis_(95ci),Additional_good_outcomes_per_1000_patients_(median),Additional_good_outcomes_per_1000_patients_(low_5%),Additional_good_outcomes_per_1000_patients_(high_95%),Additional_good_outcomes_per_1000_patients_(mean),Additional_good_outcomes_per_1000_patients_(stdev),Additional_good_outcomes_per_1000_patients_(95ci),Onset_to_needle_(mean),calibration,scenario
AGNOF1041H,258.57,235.47,287.63,258.91,15.60,3.06,15.35,13.26,17.29,15.36,...,0.23,12.97,11.10,14.69,12.97,1.13,0.22,162.39,1.0,base
AKCGO9726K,262.90,243.22,286.13,263.31,12.59,2.47,15.14,13.47,16.80,15.17,...,0.20,13.41,11.85,14.81,13.42,1.00,0.20,154.40,1.0,base
AOBTM3098N,246.00,213.90,272.00,244.42,18.97,3.72,7.80,6.40,9.41,7.90,...,0.19,5.65,4.50,7.08,5.74,0.81,0.16,182.86,1.0,base
APXEE8191H,239.18,204.90,266.63,237.56,20.19,3.96,10.25,7.96,12.07,10.00,...,0.28,7.35,5.32,9.06,7.23,1.07,0.21,180.64,1.0,base
ATDID5461S,232.73,196.18,280.00,235.27,25.64,5.03,9.09,6.55,12.02,9.22,...,0.33,6.28,4.46,8.38,6.33,1.26,0.25,188.42,1.0,base
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YPKYH1768F,280.00,232.00,320.00,277.28,28.45,5.58,21.60,18.38,26.82,21.76,...,0.51,22.94,18.89,28.51,23.10,3.02,0.59,120.78,1.0,speed_onset_benchmark
YQMZV4284N,237.43,200.98,276.68,238.69,22.71,4.45,13.97,10.89,16.77,13.92,...,0.37,12.36,9.46,14.98,12.27,1.75,0.34,140.37,1.0,speed_onset_benchmark
ZBVSO0975W,257.24,222.72,291.87,255.72,22.54,4.42,21.27,18.47,24.29,21.27,...,0.39,20.39,17.73,23.80,20.40,1.87,0.37,131.75,1.0,speed_onset_benchmark
ZHCLE1578P,244.35,220.98,268.91,244.31,14.63,2.87,13.82,12.44,15.96,14.08,...,0.22,12.95,11.22,14.86,12.94,1.13,0.22,135.52,1.0,speed_onset_benchmark


## Save results

In [19]:
results_all['stroke_team'] = results_all.index
results_all.to_csv('./output/scenario_results.csv', index=False)